In [14]:
import json
import requests
import random
import string
import secrets
import time
import collections
import re
from urllib.parse import parse_qs
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
import random
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer


try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [15]:
file_path = 'words_250000_train.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
words = content.split('\n')  # Replace with your words

In [16]:
import random

def randomly_mask_string(input_string, mask_probability):
    masked_string = ""
    for char in input_string:
        if random.random() < mask_probability:
            masked_string += "_"
        else:
            masked_string += char
    return masked_string

mask_probability = 0.6

tr1 = []

for i in range(len(words)):
        num_replacements = 1
        tr1.append(randomly_mask_string(words[i],mask_probability))

mask_probability = 0.5

tr11 = []

for i in range(len(words)):
        num_replacements = 1
        tr11.append(randomly_mask_string(words[i],mask_probability))

In [17]:
tr2 = []

def next_guess(masked,full):
    for i in range(len(masked)):
        if masked[i]=="_":
            return full[i]
    return ""
for i in range(len(words)):
    tr2.append((tr1[i],next_guess(tr1[i],words[i])))
    tr2.append((tr11[i],next_guess(tr11[i],words[i])))



In [18]:
mask_probability = 0.6

tr1 = []

for i in range(len(words)):
        num_replacements = 1
        tr1.append(randomly_mask_string(words[i],mask_probability))

mask_probability = 0.5

tr11 = []

for i in range(len(words)):
        num_replacements = 1
        tr11.append(randomly_mask_string(words[i],mask_probability))

for i in range(len(words)):
    tr2.append((tr1[i],next_guess(tr1[i],words[i])))
    tr2.append((tr11[i],next_guess(tr11[i],words[i])))

In [19]:
mask_probability = 0.6

tr1 = []

for i in range(len(words)):
        num_replacements = 1
        tr1.append(randomly_mask_string(words[i],mask_probability))

mask_probability = 0.6

tr11 = []

for i in range(len(words)):
        num_replacements = 1
        tr11.append(randomly_mask_string(words[i],mask_probability))

for i in range(len(words)):
    tr2.append((tr1[i],next_guess(tr1[i],words[i])))
    tr2.append((tr11[i],next_guess(tr11[i],words[i])))

In [20]:
tr2[:10]

[('__l_____', 'a'),
 ('aa___u__', 'l'),
 ('aa__i_', 'l'),
 ('_a___s', 'a'),
 ('aa_s_', 'l'),
 ('__lst', 'a'),
 ('___', 'a'),
 ('_a_', 'a'),
 ('___dahl', 'a'),
 ('aa__ahl', 'n')]

In [21]:
from collections import defaultdict


def analyze_all_substring_frequencies(dictionary, substring_length):
    substring_frequencies = defaultdict(int)

    for word in dictionary:
        for i in range(len(word) - substring_length + 1):
            substring = word[i:i+substring_length]
            substring_frequencies[substring] += 1


    sorted_substring_frequencies = dict(sorted(substring_frequencies.items(), key=lambda item: item[1], reverse=True))

    return sorted_substring_frequencies


substring_length = 1
substring_frequencies = analyze_all_substring_frequencies(words, substring_length)


sorted_keys_by_frequency = sorted(substring_frequencies, key=substring_frequencies.get, reverse=True)

sorted_keys_by_frequency = ['e','s','i','a','r','n','t','o','l','c','d','u','g','p','m','h','b','y','f','v','k', 'w', 'z', 'x', 'j','q']
print(f"List of Substrings Sorted by Frequency:")
print(sorted_keys_by_frequency)

List of Substrings Sorted by Frequency:
['e', 's', 'i', 'a', 'r', 'n', 't', 'o', 'l', 'c', 'd', 'u', 'g', 'p', 'm', 'h', 'b', 'y', 'f', 'v', 'k', 'w', 'z', 'x', 'j', 'q']


In [22]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = sorted_keys_by_frequency

        self.current_dictionary = []
        self.vectorizer = DictVectorizer()
        self.model = DecisionTreeClassifier()
        self.train_ml_model()

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link


    def train_ml_model(self):
        training_data = tr2
        X_train = []
        y_train = []

        for masked_word, guessed_letter in training_data:
            revealed_letters = [letter for letter in masked_word if letter != "_"]
            features = self.generate_features(revealed_letters, guessed_letter)
            X_train.append(features)
            y_train.append(guessed_letter)

        self.vectorizer = DictVectorizer()
        X_train_vectorized = self.vectorizer.fit_transform(X_train)

        self.model = DecisionTreeClassifier()
        self.model.fit(X_train_vectorized, y_train)






    def guess_ngram_based(self, revealed_letters, word, ngram_size):
        ngrams = self.generate_ngrams(self.current_dictionary, ngram_size)
        filtered_ngrams = {
            ngram: frequency for ngram, frequency in ngrams.items()
            if any(letter in ngram for letter in revealed_letters)
        }

        if not filtered_ngrams:
            return self.guess_default()

        max_frequency_ngram = max(filtered_ngrams, key=filtered_ngrams.get)
        available_letters = [letter for letter in max_frequency_ngram if letter not in self.guessed_letters]

        if available_letters:
            self.guessed_letters.append(available_letters[0])
            return available_letters[0]

        return self.guess_default()

    def guess_ml_based(self, revealed_letters, word, top_n=4):
        print("///////////////////////////////////////")
        X = self.generate_features(revealed_letters, word)
        X_vectorized = self.vectorizer.transform(X)

        predicted_probabilities = self.model.predict_proba(X_vectorized)

        available_probabilities = []
        for i, letter in enumerate(self.full_dictionary_common_letter_sorted):
            if letter not in self.guessed_letters:
                available_probabilities.append((i, predicted_probabilities[0][i]))

        if available_probabilities:
            available_probabilities.sort(key=lambda x: x[1], reverse=True)
            top_available_indices = [index for index, _ in available_probabilities[:top_n]]


            if all(self.full_dictionary_common_letter_sorted[index] in self.guessed_letters for index in top_available_indices):
                return self.guess_ngram_based(revealed_letters, word,4)


            for index in top_available_indices:
                letter = self.full_dictionary_common_letter_sorted[index]
                if letter not in self.guessed_letters:
                    self.guessed_letters.append(letter)
                    return letter

        return self.guess_default()

    def generate_ngrams(self, dictionary, n):
        print("n gram chal gya")
        ngram_frequencies = defaultdict(int)
        for word in dictionary:
            for i in range(len(word) - n + 1):
                ngram = word[i:i + n]
                ngram_frequencies[ngram] += 1

        sorted_ngram_frequencies = dict(sorted(ngram_frequencies.items(), key=lambda item: item[1], reverse=True))
        return sorted_ngram_frequencies


    def guess(self, word):
        s = ''
        for i in word:
          if i!=" ":
            s+=i
        word = s
        c = 0
        for i in word:
            if i == "_":
                c+=1
        ngram_size = 2
        print(word,len(word))
        if c==len(word):
            return self.guess_default()

        if word[-2:]=="l_":
            if 'y' not in self.guessed_letters:
                self.guessed_letters.append('y')
                return 'y'

        if len(word)>5:
            if word[-3:]=="in_":
                if 'g' not in self.guessed_letters:
                    self.guessed_letters.append('g')
                    return 'g'

        revealed_letters = [letter for letter in word if letter != "_"]
        if len(word)<5:
            return self.guess_ngram_based(revealed_letters, word, ngram_size)
        if revealed_letters:
            return self.guess_ml_based(revealed_letters, word)


    def generate_features(self, revealed_letters, word):
        feature = {'letter': None}
        for letter in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                feature['letter'] = letter
                for revealed_letter in revealed_letters:
                    feature[f'prev_{revealed_letter}'] = int(revealed_letter in word)  # Use binary encoding
                #break  # Stop after finding the first unguessed letter
        return feature


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    def guess_default(self):
        print("bola tha na")
        for i in self.full_dictionary_common_letter_sorted:
            if i not in self.guessed_letters:
                self.guessed_letters.append(i)
                return i

        return " "



    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [23]:
api = HangmanAPI(access_token="dfcf6b9bfe4669669dbf9ec1c3775d", timeout=2000)

In [24]:
l= api.my_status()

#[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes]

print(l)

[7832, 1, 0, 976]


In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)


In [35]:
l= api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = l[2]/l[1]
print(l)


[7832, 1000, 139, 976]


FINAL RESULT ON 1000 results : 14%